In [102]:
import pandas as pd
import numpy as np
import os 
import re
import ast
pd.options.display.max_columns = None

Cargaremos los dos ficheros de datos

In [103]:
df_limp = pd.read_csv("attacks_limpieza_5.csv", index_col=0)
df_limp.head(2)

,case_number,type,country,area,location,activity,name,sex_,age,time,species_,href,fatal_(y/n),injury,date,sex_mode,type_mode,year,age_new
0,1800.00.00,Unprovoked,seychelles,St. Anne,NaN,a corsair's boat was overturned,NaN,F,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, all onboard were killed by sharks",1800,F,Unprovoked,1927.133059,27.267683
1,1797.05.28.R,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,FATAL,Reported May-28-1797,M,Unprovoked,1927.133059,27.267683


In [104]:
clima = pd.read_csv("etl_1.csv", index_col = 0)
clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,11,1012,none,1,0,100,3,USA
1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,2,11,1013,none,1,0,100,4,USA


In [105]:
clima.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
            54, 55, 56, 57, 58, 59, 60, 61, 62, 63],
           dtype='int64', length=320)

 - Reseteamos el index para que no se repitan valores y así podamos hacer el siguiente paso sin errores.

In [106]:
clima.reset_index(inplace=True)

 - Eliminamos la antigua columna 'index'.

In [107]:
clima.drop('index', inplace=True, axis=1)

In [108]:
clima.shape

(320, 17)

Del dataframe de los ataques nos quedaremos solo con las filas de los paises que seleccionamos en la lección de ayer:
 - USA
 - Australia
 - New Zealand
 - South Africa
 - Papua New Guinea

In [109]:
filtrado_paises = df_limp[(df_limp['country'] == 'australia') 
                  |(df_limp['country'] == 'usa')|(df_limp['country'] == 'south africa') | (df_limp['country'] == 'papua new guinea')
                  | (df_limp['country'] == 'new zealand')]

 - Groupby de rh_profile

In [111]:
clima['rh_profile'] = clima['rh_profile'].apply(ast.literal_eval)

In [112]:
len(clima['rh_profile'][0])

16

In [113]:
rh = clima['rh_profile'].apply(pd.Series)

In [114]:
rh.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 13}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': -2}"
1,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -3}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': -1}"


In [115]:
for i in range(len(rh.columns)): 

    nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
    valores = list(rh[i].apply(pd.Series)["rh"] )
    clima.insert(i, nombre, valores)

In [116]:
clima.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,13,13,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-2,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,11,1012,none,1,0,100,3,USA
1,14,12,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,2,11,1013,none,1,0,100,4,USA


 - Groupby de wind_profile

In [117]:
clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

In [118]:
x = clima['wind_profile'].apply(pd.Series)

In [119]:
x.sample()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
58,"{'layer': '950mb', 'direction': 90, 'speed': 4}","{'layer': '900mb', 'direction': 100, 'speed': 5}","{'layer': '850mb', 'direction': 100, 'speed': 5}","{'layer': '800mb', 'direction': 100, 'speed': 4}","{'layer': '750mb', 'direction': 105, 'speed': 3}","{'layer': '700mb', 'direction': 110, 'speed': 3}","{'layer': '650mb', 'direction': 80, 'speed': 3}","{'layer': '600mb', 'direction': 60, 'speed': 3}","{'layer': '550mb', 'direction': 20, 'speed': 3}","{'layer': '500mb', 'direction': 355, 'speed': 5}","{'layer': '450mb', 'direction': 345, 'speed': 5}","{'layer': '400mb', 'direction': 340, 'speed': 5}","{'layer': '350mb', 'direction': 200, 'speed': 3}","{'layer': '300mb', 'direction': 245, 'speed': 5}","{'layer': '250mb', 'direction': 255, 'speed': 5}","{'layer': '200mb', 'direction': 260, 'speed': 7}"


In [120]:
for i in range(len(x.columns)): 

    nombre = "layer" + str(x[i].apply(pd.Series)["layer"][0]) 
    valores = list(x[i].apply(pd.Series)["speed"] )
    clima.insert(i, nombre, valores)

In [125]:
clima.head(2)

,layer950mb,layer900mb,layer850mb,layer800mb,layer750mb,layer700mb,layer650mb,layer600mb,layer550mb,layer500mb,layer450mb,layer400mb,layer350mb,layer300mb,layer250mb,layer200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,4,3,3,3,2,2,2,3,3,3,3,4,3,3,3,4,13,13,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-2,3,3,-9999,-9999,-9999,27,-1,11,1012,none,1,0,100,3,USA
1,4,4,3,3,2,2,2,2,3,3,3,4,3,3,3,3,14,12,10,2,-2,-2,-3,-4,-4,-4,-2,-1,-3,-3,-3,-1,6,4,-9999,-9999,-9999,27,2,11,1013,none,1,0,100,4,USA


 - Eliminamos las columnas rh_profile y wind_profile porque ya hemos sacado la información que necesitábamos de ellas.

In [123]:
clima.drop(['rh_profile', 'wind_profile'], axis=1, inplace=True)

 - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por pais.


In [126]:
df = clima.groupby('pais').mean()

In [129]:
df.reset_index()

,pais,layer950mb,layer900mb,layer850mb,layer800mb,layer750mb,layer700mb,layer650mb,layer600mb,layer550mb,layer500mb,layer450mb,layer400mb,layer350mb,layer300mb,layer250mb,layer200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,Australia,2.375000,2.921875,3.281250,3.453125,3.531250,3.593750,3.781250,4.046875,4.171875,4.218750,4.359375,4.609375,4.562500,4.562500,4.515625,4.812500,3.343750,1.687500,1.406250,1.593750,2.203125,2.593750,2.218750,2.437500,3.156250,2.937500,3.640625,4.343750,4.843750,6.937500,9.484375,11.156250,97.5,5.140625,-9999.0,-9999.0,-9999.0,22.609375,1.90625,5.031250,1015.468750,0.000000,0.0,136.093750,2.203125
1,New Zealand,3.859375,4.000000,4.046875,4.265625,4.531250,4.859375,5.218750,5.546875,5.890625,6.437500,7.218750,7.984375,8.734375,9.546875,10.703125,11.296875,12.500000,13.046875,10.234375,7.406250,5.640625,4.734375,4.156250,4.125000,4.296875,4.218750,4.093750,4.703125,5.093750,5.937500,5.843750,5.390625,97.5,5.156250,-9999.0,-9999.0,-9999.0,20.000000,1.90625,10.296875,1016.703125,6.656250,0.0,210.312500,3.468750
2,Papua New Guinea,4.406250,4.390625,4.500000,4.437500,4.468750,4.656250,4.843750,4.953125,5.093750,5.375000,5.593750,5.812500,6.234375,7.265625,8.218750,8.250000,13.000000,14.218750,10.453125,7.109375,6.203125,6.968750,6.531250,6.484375,6.250000,5.515625,4.156250,4.671875,5.281250,3.828125,1.343750,-1.828125,97.5,8.593750,-9999.0,-9999.0,-9999.0,8.609375,13.12500,10.406250,1010.187500,2.859375,0.0,193.281250,4.109375
3,South Africa,3.125000,2.921875,2.812500,2.640625,2.671875,2.781250,2.859375,3.093750,3.187500,3.359375,3.515625,3.734375,4.281250,4.421875,5.140625,5.656250,13.656250,12.734375,11.796875,10.375000,8.484375,8.390625,7.968750,7.015625,5.609375,4.203125,3.750000,3.515625,5.109375,6.671875,7.328125,2.859375,97.5,5.625000,-9999.0,-9999.0,-9999.0,24.093750,0.37500,10.796875,1019.156250,3.484375,0.0,133.437500,2.984375
4,USA,3.468750,3.421875,3.125000,2.781250,2.281250,2.203125,2.437500,2.609375,2.671875,3.031250,3.687500,4.000000,4.015625,4.515625,5.109375,5.234375,12.953125,11.671875,6.875000,2.500000,-0.234375,-1.328125,-1.671875,-1.468750,-1.671875,-1.437500,0.828125,-1.296875,-2.203125,-1.906250,-0.859375,2.250000,97.5,2.937500,-9999.0,-9999.0,-9999.0,26.578125,-0.53125,10.453125,1012.546875,1.921875,0.0,100.546875,3.140625


 - Guardar los resultados obtenidos en un csv.


In [130]:
df.to_csv('cinco_paises.csv')